# Read Input Datas

In [22]:
def readgraph(graph):
    f = open(graph,'r')
    num_of_test = int(f.readline().rstrip())
    probs = []
    for n in range(num_of_test):
        num_of_node, num_of_edge =  map(int,f.readline().rstrip().split())
        graph = {} # make dictionary
        for i in range(num_of_node+1):
            graph[i]={}
        for i in range(num_of_edge):
            node_s, node_t, cost = map(int, f.readline().rstrip().split())
            graph[node_s][node_t] = cost
        probs.append(graph)
    return probs

# Dijkstra Algorithm

### Priority Queue

In [2]:
class Priority_Queue:
    def __init__(self):
        self.Q=[] #Q라는 list안에 [key,val]로 들어감
        pass
    
    def parent(self,idx):
        par_idx = (idx-1)/2
        return self.Q[par_idx][0], par_idx
    
    def left_child(self,idx):
        return self.Q[2*idx+1][0], 2*idx+1 #key,idx
    
    def right_child(self,idx):
        return self.Q[2*idx+2][0], 2*idx+2
    
    def min_left_right_child(self,idx):
        if len(self.Q)-1<2*idx+2: #오른쪽 child 없는경우
            return self.left_child(idx)
        key_l,idx_l = self.left_child(idx)
        key_r,idx_r = self.right_child(idx)
        if key_l <= key_r:
            return key_l,idx_l
        else:
            return key_r,idx_r
    
    def insert(self, key, val):
        self.Q+=[[key,val]]
        n = len(self.Q)
        i=n-1
        while(True):
            if i==0:
                break
            par_key,par_idx = self.parent(i)
            if self.Q[i][0] < par_key:
                self.Q[i],self.Q[par_idx] = self.Q[par_idx],self.Q[i]
                i = par_idx
            else:
                break
    
    def deleteMin(self):
        minimum = self.Q[0]
        if len(self.Q)==1:
            return self.Q.pop()[1]
        else:
            self.Q[0] = self.Q.pop()
        i=0
        while(True):
            #자식이 없으면 break
            if 2*i+1>len(self.Q)-1: 
                break
            
            #자식들과 비교하면서 내려가기
            min_child,child_idx = self.min_left_right_child(i)
            if self.Q[i] > min_child:
                self.Q[i],self.Q[child_idx] = self.Q[child_idx],self.Q[i]
                i = min_child
            else:
                break
        return minimum[1]
    
    def decreaseKey(self,val, new_key):#idx,new_val):#val new_key
        #Q에서 node(val)를 가지고 있는 거 찾기
        for i in range(len(self.Q)):
            if self.Q[i][1] == val:
                ind = i
        
        #원래 key가 newkey보다 크면
        if self.Q[ind][0]>new_key: 
            self.Q[ind][0] = new_key
            while(True):
                if ind==0:
                    break
                par_key,par_idx = self.parent(ind)
                if self.Q[ind][0] < par_key:
                    self.Q[ind],self.Q[par_idx] = self.Q[par_idx],self.Q[ind]
                    ind = par_idx
                else:
                    break

    def is_empty(self):
        if self.inquiry_pq()==[]:
            return True
        else:
            return False
    
    def check_exist_node(self, val):
        for i in range(len(self.Q)):
            if self.Q[i][1] == val:
                return True
        return False
    
    def inquiry_pq(self):
        return self.Q

### Dijkstra algorithm using Priority Queue

In [3]:
class Dijkstra:
    def __init__(self, G, s):
        inf = float("inf")
        self.n = len(G)
        self.s = s
        self.G = G
        self.dist={}; self.parent={}
        for v in G:
            self.dist[v] = inf
            self.parent[v] = None
        self.dist[s] = 0
        self.parent[s] = s
        self.Q = Priority_Queue()
        self.Q.insert(key=self.dist[s],val=s)
    
    def dst(self):
        while not self.Q.is_empty():
            u = self.Q.deleteMin()
            for v in self.G[u]:
                if self.dist[u]+self.G[u][v] < self.dist[v]:
                    self.dist[v] = self.dist[u]+self.G[u][v]
                    self.parent[v] = u
                    if not self.Q.check_exist_node(v):
                        self.Q.insert(key=self.dist[v],val=v)
                    else:
                        self.Q.decreaseKey(v,self.dist[v])
        return self.dist, self.parent

## Bellman-Ford Algorithm

In [4]:
class BellmanFord:
    def __init__(self, G, s):
        inf = float("inf")
        self.n = len(G)
        self.G = G
        self.s = s
        self.dist={}; self.parent={}
        for v in G:
            self.dist[v] = inf
            self.parent[v] = None
    
    def relax(self,u,v):
        if self.dist[u]+self.G[u][v] < self.dist[v]:
            self.dist[v] = self.dist[u]+self.G[u][v]
            self.parent[v] = u
    
    def infect(self,v):
        if self.dist[v] > -float("inf"):
            self.dist[v] = -float("inf")
            for w in self.G[v]:
                self.infect(w)
        
    def bf(self): # s : source node
        self.dist[self.s] = 0
        self.parent[self.s] = self.s
        for i in range(self.n):
            for u in self.G:
                for v in self.G[u]:
                    self.relax(u,v)
            
        # check_negative_cycle
        for u in self.G:
            for v in self.G[u]:
                if self.dist[u]+self.G[u][v] < self.dist[v]:
                    self.infect(v)
        return self.dist, self.parent
            

# Experiment

In [38]:
def output_format(output):
    dist = output[0]
    parent = output[1]
    n = len(parent)
    for i in range(1,n):
        seq=[i,parent[i]]
        t=parent[i]
        if t is None:
            print "0 ->",i,": ",dist[i],": ","There is no route to ",i
            break
        cnt=0
        while(t!=0 and cnt<n and t is not None):
            seq.append(parent[t])
            t = parent[t]; cnt+=1
        seq.reverse()
        print "0 ->",i,": ",dist[i],": "," ".join(map(str,seq))

In [39]:
def solve_shortestPath(graph):
#     1. make problems
    probs = readgraph(graph) #0: 음수사이클x   1: 음수사이클o
    
#     2. 음수 사이클 없는 그래프(데이터셋1) Dijkstra
    print '\n2'
    dst_algo = Dijkstra(probs[0],0)
    output_format(dst_algo.dst())
    
#     3-1. 음수 사이클 없는 그래프(데이터셋1) Bellman-Ford
    print '\n3-1'
    bf_algo1 = BellmanFord(probs[0],0)
    output_format(bf_algo1.bf())
    
#     3-2. 음수 사이클 있는 그래프(데이터셋2) Bellman-Ford
    print '\n3-2'
    bf_algo2 = BellmanFord(probs[1],0)
    output_format(bf_algo2.bf())

#     4-1. 음수 사이클 없는 그래프(데이터셋3) Dijkstra
    print '\n4-1'
    dst_algo2 = BellmanFord(probs[2],0)
    output_format(dst_algo2.bf())

#     3-2. 음수 사이클 있는 그래프(데이터셋3) Bellman-Ford
    print '\n4-2'
    bf_algo3 = BellmanFord(probs[2],0)
    output_format(bf_algo3.bf())


In [40]:
solve_shortestPath('graph.txt')


2
0 -> 1 :  1 :  0 1
0 -> 2 :  5 :  0 3 2
0 -> 3 :  4 :  0 3
0 -> 4 :  inf :  There is no route to  4

3-1
0 -> 1 :  1 :  0 1
0 -> 2 :  5 :  0 3 2
0 -> 3 :  4 :  0 3
0 -> 4 :  inf :  There is no route to  4

3-2
0 -> 1 :  -inf :  3 1 4 3 1 4 3 1
0 -> 2 :  -inf :  1 4 3 1 4 3 1 2
0 -> 3 :  -inf :  4 3 1 4 3 1 4 3
0 -> 4 :  -inf :  1 4 3 1 4 3 1 4
0 -> 5 :  3 :  0 5

4-1
0 -> 1 :  -1 :  0 1
0 -> 2 :  2 :  0 1 2
0 -> 3 :  1 :  0 1 3
0 -> 4 :  1 :  0 1 4
0 -> 5 :  3 :  0 5

4-2
0 -> 1 :  -1 :  0 1
0 -> 2 :  2 :  0 1 2
0 -> 3 :  1 :  0 1 3
0 -> 4 :  1 :  0 1 4
0 -> 5 :  3 :  0 5
